<a href="https://colab.research.google.com/github/Shanmukh459/Text-summarization/blob/main/TextSummariser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install youtube_transcript_api

In [2]:
import numpy as np
import youtube_transcript_api
from youtube_transcript_api import YouTubeTranscriptApi
import nltk
nltk.download('punkt')
import re
from nltk.corpus import stopwords
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [58]:
link = "https://youtu.be/26PrgjTboVQ?si=knhjH7BGObYdLZbW"
unique_id = link.split("/")[-1][:11]

In [49]:
transcript_list = YouTubeTranscriptApi.list_transcripts(unique_id)
for transcript in transcript_list:
  print(
        transcript.video_id,
        transcript.language,
        transcript.language_code,
        # whether it has been manually created or generated by YouTube
        transcript.is_generated,
        # whether this transcript can be translated or not
        transcript.is_translatable,
        # # a list of languages the transcript can be translated to
        transcript.translation_languages,
    )
  print(transcript.fetch())
  print(transcript.translate('en').fetch())


False
False


In [62]:
link = "https://www.youtube.com/watch?v=Y8Tko2YC5hA"
link = "https://youtu.be/26PrgjTboVQ?si=knhjH7BGObYdLZbW"
unique_id = link.split("/")[-1][:11]
# print(unique_id)
transcripts = YouTubeTranscriptApi.list_transcripts(unique_id)
# sub = youtube_transcript_api.find_generated_transcript(unique_id)
sub = YouTubeTranscriptApi.get_transcript(unique_id, languages=['en', 'en-GB'])
subtitle = " ".join([x['text'] for x in sub])

In [63]:
print(subtitle)

Hello. This is 6 Minute English from BBC Learning English. I’m Neil. And I’m Beth. When I was a boy, I wanted to be a fireman when I grew up. How about you, Beth? Did you have any childhood dreams? I wanted to be an astronaut and fly to the Moon… When we’re young most of us have big dreams and plans for the future. Unfortunately, as we grow up these childhood dreams often get lost in the adult world of jobs, money, families and careers. But not for everyone… Daisy, from New Zealand, and, Herman, from Argentina are two people who decided to follow their childhood dreams. They wanted the world to become a utopia – a perfect, ideal society where everyone is happy and gets along with each other. In this programme, we’ll be hearing how Daisy and Herman made their dreams come true – not by changing the world, but by changing themselves. And, as usual, we’ll be learning some new vocabulary too. But before that I have a question for you, Beth. Following your dreams can be tough, but not follow

#Summarization with TF-IDF vectorization

In [64]:
from nltk.tokenize import sent_tokenize

# subtitle = subtitle.replace("\n", "")
sentences = nltk.tokenize.sent_tokenize(subtitle)

In [65]:
organized_sent = {k:v for v,k in enumerate(sentences)}

In [66]:
tfidf = TfidfVectorizer(strip_accents='unicode',
                        max_features=None,
                        lowercase=True,
                        token_pattern=r'w{1,}',
                        ngram_range=(1, 3),
                        use_idf=1,
                        smooth_idf=1,
                        sublinear_tf=1,
                        stop_words='english')

In [67]:
sentence_vectors = tfidf.fit_transform(sentences)
sent_scores = np.array(sentence_vectors.sum(axis=1)).ravel()

/usr/local/lib/python3.10/dist-packages/sklearn/utils/_param_validation.py:558: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['w'] not in stop_words.
  warnings.warn(


In [68]:
N = 3
top_n_sentences = [sentences[index] for index in np.argsort(sent_scores, axis=0)[::-1][:N]]

Ordering the sentences based on the order they appeared in the subtitles

In [69]:
mapped_sentences = [(sentence, organized_sent[sentence]) for sentence in top_n_sentences]

mapped_sentences = sorted(mapped_sentences, key = lambda x: x[1])
ordered_sentences = [element[0] for element in mapped_sentences]

summary = " ".join(ordered_sentences)

In [70]:
summary

'But not for everyone… Daisy, from New Zealand, and, Herman, from Argentina are two people who decided to follow their childhood dreams. They wanted the world to become a utopia – a perfect, ideal society where everyone is happy and gets along with each other. Twenty-two years and three children later they have visited over a hundred countries, meeting with countless people and experiences on the way.'

# Summarizing using BART

In [71]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 91.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.6 MB/s eta 0:00:00


In [72]:
import transformers
from transformers import BartTokenizer, BartForConditionalGeneration

In [73]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')

In [74]:
input_tensor = tokenizer.encode(subtitle, return_tensors="pt", max_length=512)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [75]:
outputs_tensor = model.generate(input_tensor, max_length=160, min_length=120, length_penalty=2.0, num_beams=4, early_stopping=True)
outputs_tensor

tensor([[    2,     0, 32259, 26351,     6,    31,   188,  3324,     6,     8,
             6, 14358,     6,    31,  5244,    32,    80,    82,    54,  1276,
             7,  1407,    49,  6585,  7416,     4,   252,   770,     5,   232,
             7,   555,    10, 16080, 26184,   126,    10,  1969,     6,  5631,
          2313,   147,   961,    16,  1372,     8,  1516,   552,    19,   349,
            97,     4,    96,    42,  3020,     6,    52,    17,    27,   890,
            28,  1576,   141, 25246,     8, 14358,   156,    49,  7416,   283,
          1528,   126,    45,    30,  2992,     5,   232,     6,    53,    30,
          2992,  1235,     4,   125,   137,    14,    38,    33,    10,   864,
            13,    47,     6, 10472,     4,  3515,   110,  7416,    64,    28,
          1828,     6,    53,    45,   511,   106,    64,   989,    47,  9917,
          2577,    70,     5,   383,    47,   770,     7,   109,    53,   399,
            17,    27,    90,     4,     2]])

In [76]:
print(tokenizer.decode(outputs_tensor[0]))

</s><s>Daisy, from New Zealand, and, Herman, from Argentina are two people who decided to follow their childhood dreams. They wanted the world to become a utopia – a perfect, ideal society where everyone is happy and gets along with each other. In this programme, we’ll be hearing how Daisy and Herman made their dreams come true – not by changing the world, but by changing themselves. But before that I have a question for you, Beth. Following your dreams can be tough, but not following them can leave you regretting all the things you wanted to do but didn’t.</s>


# Summarizing using Transformers' pipeline

In [77]:
from transformers import pipeline


In [78]:
summarizer = pipeline('summarization')


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [84]:
summary = summarizer(input_tensor, max_length=180, min_length=  30)

ValueError: ignored